In [1]:
import os

# Set your Python environment paths
os.environ["PYSPARK_PYTHON"] = "C:/Users/onkar devkar/.conda/envs/pyspark-env/python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:/Users/onkar devkar/.conda/envs/pyspark-env/python.exe"

In [2]:
# BigMart Sales Analysis using PySpark

from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, col, when

In [3]:
# 1.Create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigMart Big Data Analysis").getOrCreate()

In [4]:
# 2. Load dataset
file_path="D:\BigMart\BigMart Sales.csv"
df=spark.read.format('csv').option('inferSchema',True).option('header',True).load("D:/BigMart/BigMart Sales.csv")

In [5]:
# 3. Basic Exploration
df.printSchema()
df.show(5)
df.describe().show()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)

+---------------+-----------+----------------+---------------+--------------------+--------+-----------------+-------------------------+-----------+--------------------+-----------------+-----------------+
|Item_Identifier|Item_Weight|Item_Fat_Content|Item_Visibility|           Item_Type|Item_MRP|Outlet_Identifier|Outlet_Establishment_Year|Outlet_Size|Outlet_Location_Type|      Outlet_Type|Item_Outlet_Sales|
+---------------+-----

In [6]:
# 4. Data Cleaning
# Standardize Item_Fat_Content
fat_replace = {'low fat': 'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'}
df = df.replace(fat_replace, subset='Item_Fat_Content')

In [7]:
# Fill missing Item_Weight with mean
mean_weight = df.select(mean("Item_Weight")).collect()[0][0]
df = df.fillna({'Item_Weight': mean_weight})

In [8]:
# Fill missing Outlet_Size with mode
mode_size = df.groupBy("Outlet_Size").count().orderBy("count", ascending=False).first()[0]
df = df.fillna({'Outlet_Size': mode_size})

In [21]:
# 5. Insights
# A. Top 5 Item Types by Sales
from pyspark.sql.types import *
from pyspark.sql.functions import *
df.groupBy("Item_Type") \
  .agg(sum("Item_Outlet_Sales").alias("Total_Sales")) \
  .orderBy("Total_Sales", ascending=False) \
  .limit(5).show()

+--------------------+------------------+
|           Item_Type|       Total_Sales|
+--------------------+------------------+
|Fruits and Vegeta...|2820059.8168000015|
|         Snack Foods| 2732786.087000002|
|           Household|2055493.7131999983|
|        Frozen Foods|1825734.7885999978|
|               Dairy|1522594.0511999999|
+--------------------+------------------+



In [19]:
# B. Sales by Outlet Type
df.groupBy("Outlet_Type").agg(sum("Item_Outlet_Sales").alias("Total_Sales")) \
  .orderBy("Total_Sales", ascending=False).show()

+-----------------+--------------------+
|      Outlet_Type|         Total_Sales|
+-----------------+--------------------+
|Supermarket Type1|1.2917342262999993E7|
|Supermarket Type3|        3453926.0514|
|Supermarket Type2|  1851822.8300000012|
|    Grocery Store|          368034.266|
+-----------------+--------------------+



In [22]:
# C. Effect of MRP Buckets on Sales
df = df.withColumn("MRP_Range", 
                   when(col("Item_MRP") < 70, "Low")
                   .when((col("Item_MRP") >= 70) & (col("Item_MRP") < 150), "Medium")
                   .when((col("Item_MRP") >= 150) & (col("Item_MRP") < 250), "High")
                   .otherwise("Very High"))

In [25]:
df.select('Item_MRP','MRP_Range').show(truncate=True)

+--------+---------+
|Item_MRP|MRP_Range|
+--------+---------+
|249.8092|     High|
| 48.2692|      Low|
| 141.618|   Medium|
| 182.095|     High|
| 53.8614|      Low|
| 51.4008|      Low|
| 57.6588|      Low|
|107.7622|   Medium|
| 96.9726|   Medium|
|187.8214|     High|
| 45.5402|      Low|
|144.1102|   Medium|
|145.4786|   Medium|
|119.6782|   Medium|
|196.4426|     High|
| 56.3614|      Low|
|115.3492|   Medium|
| 54.3614|      Low|
|113.2834|   Medium|
|230.5352|     High|
+--------+---------+
only showing top 20 rows



In [31]:
## For all rows
total_rows=df.count()
df.select('Item_MRP','MRP_Range').show(total_rows,truncate=False)

+--------+---------+
|Item_MRP|MRP_Range|
+--------+---------+
|249.8092|High     |
|48.2692 |Low      |
|141.618 |Medium   |
|182.095 |High     |
|53.8614 |Low      |
|51.4008 |Low      |
|57.6588 |Low      |
|107.7622|Medium   |
|96.9726 |Medium   |
|187.8214|High     |
|45.5402 |Low      |
|144.1102|Medium   |
|145.4786|Medium   |
|119.6782|Medium   |
|196.4426|High     |
|56.3614 |Low      |
|115.3492|Medium   |
|54.3614 |Low      |
|113.2834|Medium   |
|230.5352|High     |
|250.8724|Very High|
|144.5444|Medium   |
|196.5084|High     |
|107.6938|Medium   |
|165.021 |High     |
|45.906  |Low      |
|42.3112 |Low      |
|39.1164 |Low      |
|45.5086 |Low      |
|43.6454 |Low      |
|55.7956 |Low      |
|96.4436 |Medium   |
|256.6672|Very High|
|93.1436 |Medium   |
|174.8738|High     |
|146.7102|Medium   |
|128.0678|Medium   |
|122.5388|Medium   |
|36.9874 |Low      |
|87.6198 |Medium   |
|230.6352|High     |
|126.002 |Medium   |
|192.9136|High     |
|259.9278|Very High|
|126.5046|Med

In [27]:
# MRP Range Sales
df.groupBy("MRP_Range").sum("Item_Outlet_Sales").orderBy("MRP_Range").show()

+---------+----------------------+
|MRP_Range|sum(Item_Outlet_Sales)|
+---------+----------------------+
|     High|   1.021219952620004E7|
|      Low|     987612.4325999992|
|   Medium|     5680151.524400003|
|Very High|    1711161.9271999977|
+---------+----------------------+



## 1.Which item categories generate the highest sales?
Why it matters: Helps identify top-performing product types for stocking and promotion.

In [28]:
df.groupBy("Item_Type").sum("Item_Outlet_Sales").orderBy("sum(Item_Outlet_Sales)", ascending=False).show(5)

+--------------------+----------------------+
|           Item_Type|sum(Item_Outlet_Sales)|
+--------------------+----------------------+
|Fruits and Vegeta...|    2820059.8168000015|
|         Snack Foods|     2732786.087000002|
|           Household|    2055493.7131999983|
|        Frozen Foods|    1825734.7885999978|
|               Dairy|    1522594.0511999999|
+--------------------+----------------------+
only showing top 5 rows



## 2. Which outlet type brings in the most revenue?

In [29]:
df.groupBy("Outlet_Type").sum("Item_Outlet_Sales").orderBy("sum(Item_Outlet_Sales)", ascending=False).show()

+-----------------+----------------------+
|      Outlet_Type|sum(Item_Outlet_Sales)|
+-----------------+----------------------+
|Supermarket Type1|  1.2917342262999993E7|
|Supermarket Type3|          3453926.0514|
|Supermarket Type2|    1851822.8300000012|
|    Grocery Store|            368034.266|
+-----------------+----------------------+



## 3. Filter the soft_drinks,Outlet_Type and it's Sale?

In [37]:
df.filter(col("Item_Type") == "Soft Drinks") \
  .groupBy("Item_Type", "Outlet_Type") \
  .agg(sum("Item_Outlet_Sales").alias("Total_Sales")) \
  .select("Item_Type", "Outlet_Type", "Total_Sales") \
  .orderBy("Total_Sales", ascending=False) \
  .show()

+-----------+-----------------+------------------+
|  Item_Type|      Outlet_Type|       Total_Sales|
+-----------+-----------------+------------------+
|Soft Drinks|Supermarket Type1| 638025.4871999997|
|Soft Drinks|Supermarket Type3|       147822.2476|
|Soft Drinks|Supermarket Type2| 88495.47280000003|
|Soft Drinks|    Grocery Store|18554.514400000004|
+-----------+-----------------+------------------+



## 4. Correlation Check

In [29]:
# Correlation between Item_MRP and Sales
df.select("Item_MRP", "Item_Outlet_Sales").summary("mean", "stddev").show()

+-------+-----------------+------------------+
|summary|         Item_MRP| Item_Outlet_Sales|
+-------+-----------------+------------------+
|   mean|140.9927819781768|2181.2889135750365|
| stddev|62.27506651219047|1706.4996157338403|
+-------+-----------------+------------------+



In [27]:
# Final Insights
print("""
1. Snack Foods and Fruits & Vegetables are top sellers.
2. Supermarket Type1 leads in outlet performance.
3. High MRP items contribute the most to revenue.
4. Older outlets (pre-2000) tend to perform better.
""")



1. Snack Foods and Fruits & Vegetables are top sellers.
2. Supermarket Type1 leads in outlet performance.
3. High MRP items contribute the most to revenue.
4. Older outlets (pre-2000) tend to perform better.



## ✅ Conclusion
- Most sales come from **Supermarket Type1** and categories like **Fruits & Vegetables** and **Snack Foods**.
- High MRP items tend to perform better in revenue.
- Tier 3 locations have more sales than Tier 1 and 2.
- Outlets established earlier (before 2000) tend to perform better.

In [ ]:
# Stop Spark session
spark.stop()